domingo 16 de mayo de 2019  
  
_Benjamín Hernández Cortés_ - _Juan Pablo Rojas Rojas_  
_Departamento de Ingeniería Informática (DIINF)_  
_Universidad de Santiago de Chile (USACH)_


## Laboratorio 3 - Fundamentos de Aprendizaje Profundo con Redes Neuronales
___

El presente código está orientado hacia la implementación de una red neuronal convolucional o CNN (Convolutional Neural Network), la cual será diseñada y empleada con el fin de clasificar un conjunto de 25.000 imagenes de perros y gatos, que pueden ser descargados a través del siguiente enlace: [Kaggle Cats and Dogs Dataset](https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765).


#### Importación de bibliotecas
---

Las bibliotecas a emplear son:
- **os:** Manejo de funcionalidades dependientes del sistema operativo
- **numpy:** Herramienta de computación científica, que permite trabajar a través de vectores
- **matplotlib:** Visualización gráfica de diversos datos de interés
- **tensorflow:** Diseño e implementación de la red neuronal convolucional
- **cv2:** Manejo del conjunto de imagenes
- **random:** Uso de funciones relacionadas con la aleatoriedad
- **tqdm:** Barra de progreso interactiva

#### ¡Precauciones!
---

Para asegurar el correcto funcionamiento de la herramienta, es necesario tener en cuenta las siguientes precauciones y las acciones a tomar, para el reparar algunas funcionalidades que podrían presentar problemas.

| <p style='text-align: left;'>**Error**</p>  | <p style='text-align: left;'>**Descripción**</p> |
| ------------ | ------------ |
| <p style='text-align: justify;'>`UnboundLocalError: local variable 'photoshop' referenced before assignment`</p> |  <p style='text-align: justify;'>Este mensaje de error se presenta al momento de utilizar la clase _ImageDataGenerator_ de _tensorflow.keras.preprocessing.image_. Este error puede ser reparado haciendo una leve modificación al archivo _JpegImagePlugin.py_ asociado a la biblioteca _PIL_, siguiendo los pasos indicados en este [post](https://github.com/python-pillow/Pillow/pull/3771#issuecomment-485104596).</p>|
| <p style='text-align: justify;'>`Problema al obtener/cargar las imagenes`</p> |  <p style='text-align: justify;'>El conjunto de imagenes indicado al principio de este trabajo ([Kaggle Cats and Dogs Dataset](https://www.microsoft.com/en-us/download/confirmation.aspx?id=54765)), posee un par de imagenes que no contienen información y por lo tanto, no pueden ser empleados. Las imagenes en cuestión son _/PetImages/Cat/666.jpg_ y _/PetImages/Dog/11702.jpg_. Se recomienda que elimine las imagenes anteriormente indicadas, previo a la ejecución de este archivo.</p>|

In [16]:
import os
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
import cv2

from random import shuffle
from tqdm import tqdm
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, Dropout, Activation, BatchNormalization

In [18]:
IMAGE_SIZE = 60
IMAGE_WIDTH, IMAGE_HEIGHT = IMAGE_SIZE, IMAGE_SIZE
BATCH_SIZE = 32
input_shape = (IMAGE_WIDTH, IMAGE_HEIGHT, 1)
DIR = "C:\\Users\\Familia Hernández\\Desktop\\Redes_neuronales_1-2019\\Laboratorio 3 - Redes Convolucionales\\PetImages\\Cat"


datagen = ImageDataGenerator(rescale=1.0/255.0,
                             validation_split=0.3)

# Conjunto de prueba
training_dataset = datagen.flow_from_directory('PetImages/',
                                               target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                               batch_size=100,
                                               color_mode='grayscale',
                                               class_mode='categorical',
                                               shuffle=True,
                                               subset='training')

# Conjunto de validación
validation_dataset = datagen.flow_from_directory('PetImages/',
                                                 target_size=(IMAGE_SIZE, IMAGE_SIZE),
                                                 batch_size=100,
                                                 color_mode='grayscale',
                                                 class_mode='categorical',
                                                 shuffle=True,
                                                 subset='validation')



model = Sequential()

model.add(Conv2D(32, (5, 5), activation='relu', input_shape=(60,60,1)))

model.add(MaxPooling2D(pool_size=(3, 3)))

#model.add(Activation('relu'))

model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(60,60,1)))

model.add(MaxPooling2D(pool_size=(3, 3)))

#model.add(Activation('relu'))

model.add(Flatten())

model.add(Dense(256, activation='relu'))

model.add(Dense(2, activation='softmax'))

model.compile(loss="categorical_crossentropy",
              optimizer="adam",
              metrics=["accuracy"])

model.summary()

model.fit_generator(training_dataset,
                    steps_per_epoch=len(training_dataset),
                    epochs = 5,
                    validation_steps = len(validation_dataset),
                    validation_data = validation_dataset);


Found 17500 images belonging to 2 classes.
Found 7498 images belonging to 2 classes.
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_17 (Conv2D)           (None, 56, 56, 32)        832       
_________________________________________________________________
max_pooling2d_17 (MaxPooling (None, 18, 18, 32)        0         
_________________________________________________________________
conv2d_18 (Conv2D)           (None, 14, 14, 64)        51264     
_________________________________________________________________
max_pooling2d_18 (MaxPooling (None, 4, 4, 64)          0         
_________________________________________________________________
flatten_13 (Flatten)         (None, 1024)              0         
_________________________________________________________________
dense_16 (Dense)             (None, 256)               262400    
_________________________________________________________

In [ ]:
len(training_dataset)

In [ ]:
def create_train_data():
    training_data = []
    for img in tqdm(os.listdir("PetImages/Cat")):
        label = [0]
        path = os.path.join("PetImages/Cat", img)
        print("Path => ", path)
        img = cv2.imread(path,cv2.IMREAD_GRAYSCALE)
        img = cv2.resize(img, (128,128))
        training_data.append([np.array(img),np.array(label)])
    shuffle(training_data)
    np.save('train_data.npy', training_data)
    return training_data

a = create_train_data()

In [ ]:
from PIL import JpegImagePlugin
JpegImagePlugin